In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_15481: Some(http://gw02.itversity.com:4049)

In [2]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1533622723243_15481


In [7]:
import org.apache.spark.ml.UnaryTransformer
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.types.{ArrayType, StringType, DataType}
import org.apache.spark.ml.param.{IntParam, ParamValidators}

class MyTokenizer(override val uid: String) 
    extends UnaryTransformer[String, Seq[String], MyTokenizer] 
    with DefaultParamsWritable {

  def this() = this(Identifiable.randomUID("myTokenizer"))

  val maxWords: IntParam = new IntParam(this, 
                                        "maxWords",
                                        "The max number of words to return.",
                                        ParamValidators.gtEq(0))

  def setMaxWords(value: Int): this.type = set(maxWords, value)

  def getMaxWords: Integer = $(maxWords)

  override protected def createTransformFunc: String => Seq[String] = (
    inputString: String) => {
      inputString.split("\\s").take($(maxWords))
  }

  override protected def validateInputType(inputType: DataType): Unit = {
    require(
      inputType == StringType, s"Bad input type: $inputType. Requires String.")
  }

  override protected def outputDataType: DataType = new ArrayType(StringType,
    true)
}
// this will allow you to read it back in by using this object.
object MyTokenizer extends DefaultParamsReadable[MyTokenizer]

defined class MyTokenizer
defined object MyTokenizer


In [9]:
// COMMAND ----------

val myT = new MyTokenizer().setInputCol("someCol").setMaxWords(2)

myT.transform(Seq("hello world. This text won't show.").toDF("someCol")).show(3, false)


// COMMAND ----------

+----------------------------------+--------------------------------+
|someCol                           |myTokenizer_84efc6874d3e__output|
+----------------------------------+--------------------------------+
|hello world. This text won't show.|[hello, world.]                 |
+----------------------------------+--------------------------------+



myT = myTokenizer_84efc6874d3e


myTokenizer_84efc6874d3e

In [10]:
val myT1 = new MyTokenizer().setInputCol("someCol").setMaxWords(3)

myT1.transform(Seq("hello world. This text won't show.").toDF("someCol")).show(3, false)


+----------------------------------+--------------------------------+
|someCol                           |myTokenizer_60bf7c689a3c__output|
+----------------------------------+--------------------------------+
|hello world. This text won't show.|[hello, world., This]           |
+----------------------------------+--------------------------------+



myT1 = myTokenizer_60bf7c689a3c


myTokenizer_60bf7c689a3c